# Matching 20201105

Compares a list of masses and labels generated with CompoundCalculator to an input peak list. It is convenient to open the Calculator and Match notebooks in side-by-side windows (Jupyter Lab allows this) so it is easy to update the target ion list and repeat the matching.

## Overview

The program allows for the possibility that conplex peak and target ion lists can result in multiple matches. The cell that shows the matches has a simplify mode whih shows only one match for each peak but appends a string showing the number of matches, currently the match show is the one with the shortest (simplest) label - Occam's razor - but picking based on mass error could be anther option. There is also a cell that shows peaks with redundant matches to emphasize their existence and allow adjustment of the matching or ion generation parameters if necessary.

Unmatched peaks greater than a given intenity threshold (percent base peak intensity) are shown. The idea is that this is part of interactive spectrum interpretation, i.e. once the origin of unmatched peaks is understood the ion generation parameters can be adjusted and applied to other peaks.

In order to ensure that isotope peaks stay with the monoisotopic peaks, the program only searches for 13C peaks for matched peaks. Peaks identified as isotopes can also match entries in the target ion list so that other possibilities are shown.

The peak list must be tab-delimied and have mass values but can also contain columns for Retention Time (RT) and Intensity; the function that reads the peak list tries to determine which columns are present.

Results can be saved in several ways including a simple mass/intensity list and more detailsd lists. The former is useful with PeakView which allows tezt lists to be imported as spectra and overlaid on the original data to vislauize the matches and highlight unmatched peaks.

## Imports and function definitions

In [1]:
import os
import datetime 
from collections import namedtuple
from itertools import groupby

In [2]:
# Define some named tuples
Peak = namedtuple('Peak', 'Mass Inten RT')
Target = namedtuple('Target', 'Mass Root Label')
Match = namedtuple('Match', 'Pk_index TargetIon')   # a match contaisnt the index of the peak and the calculated ion it matches

In [3]:
def values_from_line(line):
    """
    Split a line into parts and try to convert them to numbers...return the list of numbes or field
    """
    
    parts = line.split()
    
    try:
        vals = [float(field) for field in parts]  # convert all to numbers
        success = True
    except:
        vals = parts      # return the fields if the conversion fails
        success = False
    
    return success, vals
    
    
def read_peak_list(peak_file_path):
    """
    Reads a tab-delimited text file generating a list or Peak tuples (Mass, Inten, RT). Mass must be present but the other fields are optional
    and will be stored internally as zero
    If the file has only one column it is assumed to contain masses otherwise the code assumes that the first column is Mass,
    the second is Inten and the RT is absent.
    If the file contains a header line it is used to define the order of the columns by looking for matches with common labels
    e.g. mz, m/z, Mass, etc. for masses. The RT column can only be used via a header line containi 'RT' or 'rt'
    """
    mass_col = 0
    inten_col = 1
    rt_col = -1
    has_rt = False
    has_inten = True
    start_line= 0
    
    peaks = []    #list of (mass, inten, RT) tuples

    # read all the lines so we can process them one-by-one
    with open(peak_file_path, 'r') as f:  
    
        lines = f.readlines()
        
        f.close()
    
    success, vals = values_from_line(lines[0])  # try to convert the first line
    
    if not success:   # probably a header....

        for col_index, col in enumerate(vals):
            if col in ['mz', 'm/z', 'mass', 'Mass']: mass_col = col_index
            if col in ['Int', 'Inten', 'inten']: inten_col = col_index
            if col in ['RT', 'rt']: rt_col = col_index

        if rt_col > -1: has_rt = True
        if inten_col > -1: has_inten = True

        print('m:', mass_col, 'Int:', inten_col, 'RT:', rt_col)

    # Process line by line. Lines that cannot be converted to numbers are reported unless it is the first line which is then assumed
    # to be a header line
    for line in lines[start_line:]:        

        # get a list of numbers from the fields in the line - success will be false if this fails and vals will be the actual text parts
        success, vals = values_from_line(line)  
               
        if success:
            mass = vals[mass_col]
            rt = vals[rt_col] if has_rt else 0
            inten = vals[inten_col] if has_inten else 0
            
            p = Peak(mass, inten, rt)
              
            peaks.append(p) 
            
        else:
            print('Problem in line:', line, vals)   # vals will be the list of fields if there's a problem
    
    peaks = sorted(peaks, key = lambda x: x.Mass)     # ensure the list is sorted by mass

    masses, intens, rts = zip(*peaks)

    # Note: intensity params will be 0 if there is no intensity column
    return peaks, sum(intens), max(intens), has_rt    # peaks, tic, base_peak_inten, sum rts

In [4]:
def read_ion_list(ion_file_path):
    """
    Read the taget ion file which is assumed to contain fields for mass, root and label that can be convertes to a Target
    """
    
    ions = []            #list of (mass, root, label) tuples
    cond_str = ""        # conditions line if ptrsdrnt
    
    with open(ion_file_path, 'r') as f:  
    
        for line in f:
            
            if line[0] == '#':             # if the first character is '#' this is a condition line
                cond_str = line[1:]
                continue
            
            parts = line.split()
            
            try:
                ion = Target(float(parts[0]), parts[1], parts[2])
                ions.append(ion) 
            except:
                print('Problem in', line)

        f.close()
    
    ions = sorted(ions, key = lambda x: x.Mass)     # ensure the list is sorted by mass

    return ions, cond_str    # target ions

In [5]:
def get_match_stats(matches, peaks, tic):
    """
    Given lists of peaks and matches, determine which peaks are not matched, i.e. the index isn't in the match list,
    and return these and the sum of the intensities
    """

    matched_indices = set([m.Pk_index for m in matches])   # get the peak indices

    matched_inten = sum([peaks[i].Inten for i in matched_indices])  # sum the intensities

    percent_matched = matched_inten * 100/tic
    
    return matched_indices, percent_matched

In [6]:
def get_redundant_matches(matches):
    """
    Find which peaks have multiple matches.
    We group the matches by peak index and then find which groups have more than one entry
    We return the count and the redundant groups individually sorted by label length
    """
    redundant = []
    redundant_peak_count = 0   # Number of peaks with more than one match
    
    m_sorted = sorted(matches, key=lambda x: x.Pk_index)
    m_grps = groupby(m_sorted, lambda x: x.Pk_index)
    
    for k, grp in m_grps:
        
        # get the group as a list sorted by length of label
        grp_as_list = list(grp)
        
        if len(grp_as_list) > 1:
            redundant += sorted(grp_as_list, key= lambda x: len(x[1].Label))
            redundant_peak_count += 1
        
    return redundant_peak_count, redundant
        

In [7]:
def get_unmatched_indices(matched_indices, peaks, threshold):
    """
    Find the indices of unmatched peaks that exceed a threshold
    """    
    # get a boolean list indicating which peak is matched
    peak_matches = [True if i in matched_indices else False for i in range(len(peaks))]
    
    # now find which ones ae false
    unmatched = [i for i in range(len(peaks)) if not peak_matches[i]]
    
    return [i for i in unmatched if peaks[i][1] >= threshold]

In [8]:
def save_matches_to_file(out_path, matches_to_save, peaks, conditions):
    """
    Save the list of matches to a file. We use one of the print functions to do this"""
    with open(out_path, 'w') as f: 
        
        if conditions:
            print(conditions, file=f)
            
        print(match_short_with_tabs_header(), file=f)

        for m in matches_to_save:
            
            print(match_as_short_str_with_tabs(m, peaks), file=f)
            
        f.close()

    return(len(to_save))


In [56]:
# Functions that convert a match to various output strings
def match_as_str(m, peaks):
    """
    Prettified string for printing in a Notebook cell
    """    
    p_index, ion = m         # Unpack the peak index and the matching composition
    p = peaks[p_index]       # peak mass and intensity
    
    delta = (p.Mass - ion.Mass) * 1000   # error in mmu   
    
    return f'{p_index:5}:{p.Mass:10.4f} ({delta:5.1f}) {p.RT:6.2f} {p.Inten:12.1f} {ion.Mass:10.4f}  {ion.Root:16}{ion.Label}'

def match_as_str_with_tabs(m, peaks):
    """
   Tab delimited detailed string.
    """
    
    p_index, ion = m         # Unpack the peak index and the matching composition
    p = peaks[p_index]       # peak mass and intensity
    
    delta = (p.Mass - ion.Mass) * 1000   # error in mmu 
        
    return f'{p_index}\t{p.Mass:.4f}\t{delta:.1f}\t{p.RT:.1f}\t{p.Inten:.1f}\t{ion.Mass:.4f}\t{ion.Root}\t{ion.Label}' 

def match_with_tabs_header():
    
    return "Pk_index\tPk_mass\tDelta_mmu\tPk_RT\tPk_inten\Match_mass\Match_root\tMatch_label"

def match_as_short_str(m, peaks):
    """
    Simplified pretty string
    """
    p_index, ion = m         # Unpack the peak index and the matching composition
    p = peaks[p_index]       # peak mass and intensity
        
    return f'{p.Mass:10.4f} {p.Inten:12.1f} {p.RT:6.2f} {ion.Root} {ion.Label}'

def match_as_short_str_with_tabs(m, peaks):
    """
    Tab delimited mass and intensity for use elsewhere
    """
    p_index, ion = m         # Unpack the peak index and the matching composition
    p = peaks[p_index]       # peak mass and intensity
        
    return f'{p.Mass:.4f}\t{p.Inten:.1f}\t{p.RT:.2f}\t{ion.Root}\t{ion.Label}'

def match_short_with_tabs_header():
    
    return "Pk_mass\tPk_inten\tRT\tRoot\tMatch_label"

In [57]:
def print_match_list(matches, peaks, print_fn, simplify=False):    #provide the index of the mass field
    """
    Use the provided print function (e.g. match_as_str) to print the matches provided
    If simplify == True, we only print one entry and append a string to indicate there are more
    """
    m_sorted = sorted(matches, key=lambda x: x.Pk_index)
    m_grps = groupby(m_sorted, lambda x: x.Pk_index)
    
    for k, grp in m_grps:
        
        # get the group as a list sorted by length of label
        grp_as_list = sorted(list(grp), key= lambda x: len(x[1].Label))
        
        for g in grp_as_list:
 
            skip = False
    
            desc = print_fn(g, peaks)   
                
            if simplify and (len(grp_as_list) > 1):
                desc += f' [1/{len(grp_as_list)}]'
                skip= True
            
            print(desc)
            
            if skip: break
        
    print()


In [58]:
# Convert a list of limits, i.e. (comp, max count) tuples, to a string
# skip any with max_count = 0 and join the rest with commas
def limits_as_string(limits):
    non_zero_limits = [l for l in limits if l[1] > 0]
    if len(non_zero_limits) == 0:
        return ""
    else:
        desc = ",".join([f'{l}' for l in non_zero_limits])
        return desc

## Setup

In [70]:
# define the path for data files
# This allows the Calculator and Match notebooks to easily share data
try:
    from google.colab import drive
    
    print('Using Colab')
    
    drive.mount('/content/drive')

    data_path = os.sep + os.path.join('content', 'drive', 'MyDrive', 'SharedData')
except:
    print('Not using Colab')
    data_path = os.sep + os.path.join('Users','ronbonner','Data', 'SharedData')

file_path = os.path.join(data_path, '200325 SJ C18 Pos A, C001-U.txt')

print(file_path)


Not using Colab
/Users/ronbonner/Data/SharedData/200325 SJ C18 Pos A, C001-U.txt


In [71]:
save_matches = True                # do we want to save the matched peaks (as mass, inten, match name)?
local_files = False
include_large_unmatched = False      # do we want to include the larger unmatched peaks (by default > 1% base peak inten)
include_date_in_file_name = False

peak_half_window = 0.005     # half window for peak matching

c13_half_window = 0.005     # for matching 13C isotopes
max_C13_count = 4           # maximum number of 13C's to look for
c13_rt_window = 0.2         # main matched peaks and isotopes must have RTs that differ by less than this

In [72]:

# peak_file = "201023 Erngren guanosine peaklist.txt"

peak_file = "200325 SJ C18 Pos A, C001-U.txt"   

peak_file_path = peak_file if local_files else os.path.join(data_path, peak_file)

peaks, tic, base_peak_inten, has_RT = read_peak_list(peak_file_path)

rt_str = "has rt" if has_RT else ""

print(peak_file_path)
print(f'{len(peaks)}, peaks read. TIC {tic}, base peak inten {base_peak_inten} {rt_str}')

# for p in peaks[:10]:
#     print(p)

m: 0 Int: 2 RT: 1
Problem in line: mz	rt	Inten
 ['mz', 'rt', 'Inten']
/Users/ronbonner/Data/SharedData/200325 SJ C18 Pos A, C001-U.txt
540, peaks read. TIC 372780.0999999997, base peak inten 49781.4 has rt


In [89]:
# Set up fie names and paths...this is platform independent (i.e. we don't need to know the separator character)

# ion_file = 'Guan_CH2O2 ions pos.txt'
# compounds_as_string = 'Guan_CH2O2'      

ion_file = 'Ibu_x544 ions pos.txt'
compounds_as_string = 'Ibu_544'      

ion_path = ion_file if local_files else os.path.join(data_path, ion_file)

ions, conditions = read_ion_list(ion_path)

print(ion_path)

print(f'{len(ions)} target ions read')
if conditions: print(conditions)


/Users/ronbonner/Data/SharedData/Ibu_x544 ions pos.txt
4471 target ions read
 Time:201119_074232;Compounds:Ibu_x544;Multimer_limit:2;Polarity:positive;Phase_1:('OH', 2),('COOH', 1);Phase_2:('gluc', 1);Adducts:('Na-H', 3),('K-H', 1),('NH3', 1),('C2H4O2', 2);Losses:('H2O', 2),('HCOOH', 1)



## Match ions

We first match the ions generated by the calculator. In a subsequent step we look specifically for the 13C forms of matched peaks.

In [90]:
# Ge the current time for use in the peak name and conditions
current_time = datetime.datetime.now().replace(microsecond=0)
curr_time_str = current_time.strftime('%y%m%d_%H%M%S')

peak_index, ion_index, peaks_matched = 0, 0, 0

matches = []   # this is going to end up as a list of Match tuples : (peak index, matched target)

# Loop all the values and peaks looking for matches within the specified window
while (ion_index < len(ions)) and (peak_index < len(peaks)):

    this_peak, this_ion = peaks[peak_index], ions[ion_index]
    low_peak, high_peak = this_peak.Mass-peak_half_window, this_peak.Mass+peak_half_window
  
    # Increment the ion index if its Mass is too low and the peak if it's too high
    if this_ion.Mass < low_peak:
        ion_index += 1
        continue

    if this_ion.Mass > high_peak:
        peak_index += 1
        continue

    # save peak index and ion composition
    # since there may be more than one peak that matches this ion value, we look ahead at the peaks
    # using a separate index so the current peak can be used with the next ion value
    # we also track the ions matched since some ions may have more than one matching peak

    matches.append(Match(peak_index, this_ion))    # reference to peak and this composition
    peaks_matched += 1   
    
    look_ahead = peak_index + 1
 
    # look ahead at the peaks while they're still within the search window and add any matches to the list
    while (look_ahead < len(peaks)):
                
        look_ahead_peak = peaks[look_ahead]
        
        if(look_ahead_peak.Mass - this_ion.Mass) > peak_half_window:
            break
            
        matches.append(Match(look_ahead, this_ion))
        look_ahead +=1
        peaks_matched += 1 


    ion_index += 1 # increment ion index but not peak_index - there may be more than one ion within the window..

matched_indices, percent_tic_matched = get_match_stats(matches, peaks, tic)

matched_indices = sorted(matched_indices)
initial_matches = f'{len(matched_indices)} peaks matched ({percent_tic_matched:.1f}% tic), {len(matches)} total matches from {len(peaks)} peaks'
print(initial_matches)

171 peaks matched (71.0% tic), 197 total matches from 540 peaks


In [91]:
# Look for C13 isotopes of matched peaks

c13_matches = []

last_matched_mass = 0
last_peak_index = -1

# make sure the peaks are in mass order
matches = sorted(matches, key=lambda x: x.Pk_index)

for m in matches:    
        
    if m.Pk_index == last_peak_index:    #only need to look at each peak once
        continue

    peak_index = m.Pk_index    
    last_peak_index = peak_index
        
    m_mass, m_inten, m_rt = peaks[peak_index]     # get the matched peak...
    
    next_peak_index = peak_index      #...and start looking for isotopes at the next higher peak
        
    keep_going = True
    
    for c13_count in range(1, max_C13_count+1):  #look for 1,2,3... C13
    
        c13_mass = m_mass + (c13_count * 1.003)   # expected c13 mass
        c13_name = f'{m_mass:.4f}(+{c13_count})'  # name is based on mono mass with (+1) etc apended
        
        while next_peak_index < len(peaks) - 1:   # -1 since we're going to increment it
            
            next_peak_index += 1  # point at next value in peak list
                
            next_peak_mass, next_peak_inten, next_peak_rt = peaks[next_peak_index]
            
            # mass is out of range - 
            if next_peak_mass > (c13_mass + c13_half_window):
                keep_going = False       # when one isotope is not matched we abort and stop looking for more
                break
            
            # if the mass is in range we also check that the RT is within a window...
            # Note: it's OK to always apply this test since if there is no RT the values will be zero
            # and therefore the delta will be less than the threshold
            if next_peak_mass > (c13_mass - c13_half_window) and (abs(m_rt-next_peak_rt)< c13_rt_window):
                c13 = Target(c13_mass, m.TargetIon.Root, c13_name)
                c13_matches.append(Match(next_peak_index, c13))
                break     # and look for the next higher isotope
        
        if not keep_going:
            break     # leave 13c for loop   

# for c13m in c13_matches:
#     print(c13m)

matches += c13_matches

matches = sorted(matches, key = lambda x: x.Pk_index)  # sort by peak index...

matched_indices, percent_tic_matched = get_match_stats(matches, peaks, tic)

after_13c_match = f'{len(matched_indices)} peaks matched ({percent_tic_matched:.1f}% tic), {len(matches)} total matches from {len(peaks)} peaks'
print(after_13c_match)

238 peaks matched (91.8% tic), 281 total matches from 540 peaks


In [92]:
print_match_list(matches, peaks, match_as_str, simplify=True)

   11:  161.1317 ( -0.8)   8.73        209.3   161.1325  Ibu             Ibu-HCOOH.H+
   16:  177.1270 ( -0.4)   6.29        642.6   177.1274  Ibu_OH          Ibu_OH-HCOOH.H+
   17:  177.1271 ( -0.3)   7.82        412.0   177.1274  Ibu_OH          Ibu_OH-HCOOH.H+
   22:  183.1126 ( -1.8)   4.49        227.1   183.1144  Ibu             Ibu-HCOOH.Na-H.H+
   23:  184.1155 ( -0.1)   4.49         24.8   184.1156  Ibu             183.1126(+1)
   31:  191.1064 ( -0.3)   7.63        335.7   191.1067  Ibu_COOH        Ibu_COOH-HCOOH.H+
   38:  205.1219 ( -0.4)   8.36        403.2   205.1223  Ibu_OH          Ibu_OH-H2O.H+
   39:  205.1220 ( -0.3)   6.80        468.7   205.1223  Ibu_OH          Ibu_OH-H2O.H+
   40:  205.1220 ( -0.3)   7.60       1715.0   205.1223  Ibu_OH          Ibu_OH-H2O.H+
   41:  206.1253 (  0.4)   8.36         55.9   206.1249  Ibu_OH          205.1219(+1)
   42:  206.1256 (  0.6)   7.60        267.9   206.1250  Ibu_OH          205.1220(+1)
   44:  206.1532 ( -0.8)  10.30    

In [93]:
redundant_peak_count, redundant_matches = get_redundant_matches(matches)

print(redundant_peak_count,' redundant peaks')

for m in redundant_matches:
    print(match_as_str(m, peaks))

40  redundant peaks
  112:  260.0969 ( -0.3)   7.63        800.4   260.0972  Ibu_COOH        259.0942(+1)
  112:  260.0969 ( -2.9)   7.63        800.4   260.0998  Ibu_OH          Ibu_OH-HCOOH.(Na-H)3.NH3.H+
  116:  261.1100 (  0.2)   6.09        493.9   261.1098  Ibu_(OH)2       Ibu_(OH)2.Na-H.H+
  116:  261.1100 ( -2.1)   6.09        493.9   261.1121  Ibu_COOH        Ibu_COOH-(H2O)2.C2H4O2.H+
  144:  283.0914 ( -0.3)   6.09         58.2   283.0917  Ibu_(OH)2       Ibu_(OH)2.(Na-H)2.H+
  144:  283.0914 ( -2.7)   6.09         58.2   283.0941  Ibu_COOH        Ibu_COOH-(H2O)2.Na-H.C2H4O2.H+
  156:  292.1321 (  1.1)   7.67         51.3   292.1310  Ibu_OH          Ibu_OH-HCOOH.K-H.NH3.C2H4O2.H+
  156:  292.1321 (  3.7)   7.67         51.3   292.1284  Ibu             Ibu-(H2O)2.(Na-H)2.NH3.C2H4O2.H+
  187:  334.1087 (  0.6)  11.58         37.1   334.1081  Ibu             333.1051(+1)
  187:  334.1087 (  3.6)  11.58         37.1   334.1051  Ibu_COOH        Ibu_COOH-H2O.K-H.NH3.C2H4O2.H+
  198

In [94]:
threshold_percent = 1

bpi_percent_thresh = threshold_percent * base_peak_inten / 100

unmatched_indices = get_unmatched_indices(matched_indices, peaks, bpi_percent_thresh)  # get the unmatched peask > 1% base peak

unmatched_mass, largest_unmatched_inten = 0,0

for pi in unmatched_indices:
    m,inten,rt = peaks[pi]
    percent_base_peak = inten * 100/ base_peak_inten
    print(f'{m:10.4f} {rt:8.2f} {inten:10.0f} {percent_base_peak:8.2f}% base peak')
    
    if inten > largest_unmatched_inten:
        unmatched_mass, largest_unmatched_inten = m,inten

large_inten_rel = largest_unmatched_inten * 100/ base_peak_inten

largest_unmatched_string = f'Largest unmatched {unmatched_mass}, {largest_unmatched_inten} {large_inten_rel:.1f}% base'

print(largest_unmatched_string)

  248.1494     2.01       1128     2.27% base peak
  258.1526     5.59        912     1.83% base peak
  350.2329     8.32       2709     5.44% base peak
  351.2363     8.32        748     1.50% base peak
  398.9490     8.71        599     1.20% base peak
  407.2394     1.27       2276     4.57% base peak
  408.2428     1.27        578     1.16% base peak
  414.8938     6.18        621     1.25% base peak
  535.2740     5.59       1516     3.04% base peak
  775.2652     7.65        700     1.41% base peak
Largest unmatched 350.2329, 2708.6 5.4% base


In [95]:
# Save the match list if needed
# Note: The peak_field_path will already reflect the setting of 'local_files' so 
# we don't need to test it again

if save_matches:

    out_path, _ = os.path.splitext(peak_file_path)    # path without extension

    if include_date_in_file_name:
        out_path = f'{out_path} {compounds_as_string} matches {curr_time_str}.txt'
    else:
         out_path = f'{out_path} {compounds_as_string} matches.txt'       
    
    to_save = matches
    
    if include_large_unmatched:
        to_save = matches + [(i, Target(0,'', 'None')) for i in unmatched_indices]  # append a list of the unmatched ions as empty matches
    
    to_save = sorted(to_save, key = lambda x: x[0])  # was x[1] peak mass, now x[0], peak index

    lines_written_count = save_matches_to_file(out_path, to_save, peaks, conditions)
    
    print(out_path)
    print(lines_written_count, ' lines written')

/Users/ronbonner/Data/SharedData/200325 SJ C18 Pos A, C001-U Ibu_544 matches.txt
281  lines written


In [96]:
# summarize matches

print (current_time)

print('Compounds:', compounds_as_string)

print(len(ions), 'targets')
print()
print(f'Peaks file: {peak_file_path}')
print(f'{len(peaks)} peaks. Matching with half window {peak_half_window} amu')
print(f'Looking for <= {max_C13_count} 13C isotopes with half window {c13_half_window}')

print(initial_matches)
print('After 13C match',after_13c_match)

print(f'{len(unmatched_indices)} unmatched peaks gt {threshold_percent}%, {largest_unmatched_string}')

2020-11-19 07:42:55
Compounds: Ibu_544
4471 targets

Peaks file: /Users/ronbonner/Data/SharedData/200325 SJ C18 Pos A, C001-U.txt
540 peaks. Matching with half window 0.005 amu
Looking for <= 4 13C isotopes with half window 0.005
171 peaks matched (71.0% tic), 197 total matches from 540 peaks
After 13C match 238 peaks matched (91.8% tic), 281 total matches from 540 peaks
10 unmatched peaks gt 1%, Largest unmatched 350.2329, 2708.6 5.4% base
